# 딥 러닝을 이용한 자연어 처리 입문

아래 링크의 E-book을 보고 실습한 내용입니다.

WikiDocs 주소: https://wikidocs.net/31766


# 2장 텍스트 전처리

## 8절 One-hot encoding

## One-hot encoding

문자를 컴퓨터가 잘 처리할 수 있는 숫자 형태로 바꾸는 기법 중 하나이다. 

One-hot encoding에서는 단어 집합의 단어들을 벡터로 바꾸는 기법으로 여기서 단어 집합이란 서로 다른 단어들의 집합을 말한다. 단어 집합에서는 표기법이 다른 경우(ex. 'book', 'books')에는 서로 다른 단어로 간주한다. 

벡터로 만드는 과정은 간단하게 표현하고 싶은 단어의 인덱스에만 1을 부여하고 나머지에는 0을 부여하면 된다. 단어의 인덱스가 필요하기 때문에 텍스트의 모든 단어들을 중복을 허용하지 않고 모아 단어 집합을 만들고 정수 인코딩을 하는 과정이 필요하다.

### One-hot encoding의 한계

- 단어 집합의 크기가 곧 벡터의 차원이 되기 때문에 저장 공간 측면에서 매우 비효율적인 방법이다. 
- 단어의 유사도를 표현하지 못한다. (ex. 강아지와 늑대, 고양이와 호랑이가 유사함을 표현할 수 없다.)


In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer


text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."
text = sent_tokenize(text)

sentences = []
stop_words = set(stopwords.words("english"))

for i in text:
    # 문장 토큰화
    sentence = word_tokenize(i)
    result = []
    for word in sentence:
        # 소문자화
        word = word.lower()
        # 불용어 제거
        if word not in stop_words:
            if len(word) > 2:
                result.append(word)
    sentences.append(result)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

print(tokenizer.word_counts)


OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


### One-hot encoding 하기

In [2]:
# one hot encoding
def one_hot_encoding(word, word2Index):
    """
    @inputs
        word: array of string
        word2Index: dictionary of word, frequency pairs
    """
    # 0은 예약된 index
    one_hot_vector = [0] * len(word2Index)
    one_hot_vector[word2Index[word.lower()]] = 1
    return one_hot_vector
                            
word2index={}
for voca in sum(sentences, []):
    voca = voca.lower()
    if not word2index.get(voca):
        word2index[voca]=len(word2index)
print(word2index)

one_hot = one_hot_encoding("huge", word2index)
print("\nOne-hot encoding 결과: ", one_hot)

{'barber': 2, 'person': 1, 'good': 2, 'huge': 3, 'knew': 4, 'secret': 5, 'kept': 6, 'word': 7, 'keeping': 8, 'driving': 9, 'crazy': 10, 'went': 11, 'mountain': 12}

One-hot encoding 결과:  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]



### Keras로 One-hot encoding 하기

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# 정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)

# 문장을 인코딩
encoded = tokenizer.texts_to_sequences(text)
print("\n인코딩 된 문장: ", encoded)

# one hot encoding
one_hot = to_categorical(sum(encoded, []))
print("\nOne-hot encoding 결과:\n", one_hot)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}

인코딩 된 문장:  [[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]

One-hot encoding 결과:
 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 